We first Load the necessary dataframes and libraries

In [2]:
import pandas as pd
import numpy as np

In [ ]:
#load the data
character_metadata = pd.read_csv('./data/raw_data/character.metadata.tsv', sep='\t')

,975900,/m/03vyhn,2001-08-24,Akooshay,1958-08-26,F,1.62,Unnamed: 7,Wanda De Jesus,42,/m/0bgchxw,/m/0bgcj3x,/m/03wcfv7
184640,6764425,/m/0gmr96,1936-04-30,NaN,1895-02-04,M,1.829,NaN,Nigel Bruce,NaN,/m/02vd2xl,NaN,/m/02l99f
395866,6443785,/m/0g5mp9,2000,NaN,1935-01-22,M,NaN,NaN,Seymour Cassel,64.0,/m/03lq8k6,NaN,/m/04yyhw
34594,30386848,/m/0g5q57t,1972-05-25,NaN,NaN,NaN,NaN,NaN,Koshiro Shikano,NaN,/m/0gcshyp,NaN,/m/0gcshvg
192594,6004205,/m/0fk5s3,1999-04,Diana Carey-Lewis,1946-05-01,F,1.730,/m/0g96wd,Joanna Lumley,NaN,/m/0h2vrq9,/m/0h2vrqc,/m/0mbhr
84040,5379346,/m/0dj6zj,1993,Krishna,1954-11-07,M,1.650,/m/0cmdl5l,Kamal Haasan,38.0,/m/0k0bm5,/m/0h0yc78,/m/01x2tm8
398789,31953707,/m/0gvswz4,1996-09-13,NaN,1937-03-10,M,1.670,/m/0xnvg,Joe Viterelli,59.0,/m/0cgc6gj,NaN,/m/02h4_r
45267,12575892,/m/02wwgn7,2001-11-23,NaN,1988-10-05,M,NaN,NaN,Bobby Edner,13.0,/m/03l1gpd,NaN,/m/063gtd
443415,407351,/m/024lt6,1998-09-17,Student,1981-09-16,F,1.700,/m/01g7zj,Alexis Bledel,17.0,/m/0gyfc56,/m/0gyfc5b,/m/0217kz
319246,1722535,/m/05r4k7,1987-05,NaN,1940,M,NaN,NaN,Joe Unger,NaN,/m/0cg573h,NaN,/m/08qhs2
70344,26282276,/m/0b77p2l,1983,NaN,NaN,F,NaN,NaN,Santhakumari,NaN,/m/0gggzv6,NaN,/m/0g9kb2_


In [6]:
#rename columns
new_column_names = [
    "Wikipedia_movie_ID",
    "Freebase_movie_ID",
    "Movie_release_date",
    "Character_name",
    "Actor_date_of_birth",
    "Actor_gender",
    "Actor_height_m",
    "Actor_ethnicity",
    "Actor_name",
    "Actor_age_at_movie_release",
    "Freebase_character_actor_map_ID",
    "Freebase_character_ID",
    "Freebase_actor_ID"
]
character_metadata.columns = new_column_names

In [18]:
columns_to_check = ['Wikipedia_movie_ID','Actor_ethnicity']
remaining_rows = {col: character_metadata[col].dropna().shape[0] for col in columns_to_check}
for col, count in remaining_rows.items():
    print(f"Remaining rows for '{col}': {count}")
filtered_character = character_metadata[['Wikipedia_movie_ID', 'Actor_ethnicity']].dropna(subset=['Actor_ethnicity']) # no missing value remaining in each col

Remaining rows for 'Wikipedia_movie_ID': 450668
Remaining rows for 'Actor_ethnicity': 106058


In [8]:
def fb_to_label(freebase_id,conversion_table):
    if freebase_id in conversion_table.index:
        return conversion_table.loc[freebase_id,'label']
    else:
        return None

In [9]:
#import the file, setting freebase_id as index allows us to use .loc later
fb_wiki_gen = pd.read_csv('./data/raw_data/fb_wiki_mapping.tsv',sep='\t')
fb_wiki_gen.set_index('freebase_id',inplace=True)

In [19]:
#make list of different existing ethnicities
ethnicities_list = filtered_character['Actor_ethnicity'].unique().tolist()
#now select those from the fb_wiki_gen
fb_wiki_ethnic = fb_wiki_gen.loc[fb_wiki_gen.index.isin(ethnicities_list)]

In [20]:
#we can now change the Actor_ethnicity column from freebase_id to label
filtered_character['Actor_ethnicity']=filtered_character['Actor_ethnicity'].apply(fb_to_label,conversion_table=fb_wiki_ethnic)
#some freebase_ids couldn't be found, so we get None. We will now drop those None values
filtered_character = filtered_character.dropna(subset=['Actor_ethnicity'])

In [30]:
filtered_character.to_csv('./data/processed_data/actor_ethnicities.csv',index=False)